###Name:Poushik Yelne
###Roll No.51
### Practical No.4

####Aim:Write a program to validate a string using the parsing table generated in practical 3. Consider valid &amp; invalid string &amp; print the stack &amp; buffer input tracing.

In [ ]:
#first
def first(string):
    first_ = set()
    if string in non_terminals:
        alternatives = productions_dict[string]
        for alternative in alternatives:
            first_2 = first(alternative)
            first_ = first_ |first_2

    elif string in terminals:
        first_ = {string}
    elif string=='' or string=='#':
        first_ = {'#'}
    else:
        first_2 = first(string[0])
        if '#' in first_2:
            i = 1
            while '#' in first_2:
                first_ = first_ | (first_2 - {'#'})
                if string[i:] in terminals:
                    first_ = first_ | {string[i:]}
                    break
                elif string[i:] == '':
                    first_ = first_ | {'#'}
                    break
                first_2 = first(string[i:])
                first_ = first_ | first_2 - {'#'}
                i += 1
        else:
            first_ = first_ | first_2

    return  first_

In [ ]:
# follow cal
def follow(nonTerminal):
    follow_ = set()
    prods = productions_dict.items()
    if nonTerminal==starting_symbol:
        follow_ = follow_ | {'$'}
    for nt,rhs in prods:
        for alt in rhs:
            for char in alt:
                if char==nonTerminal:
                    following_str = alt[alt.index(char) + 1:]
                    if following_str=='':
                        if nt==nonTerminal:
                            continue
                        else:
                            follow_ = follow_ | follow(nt)
                    else:
                        follow_2 = first(following_str)
                        if '#' in follow_2:
                            follow_ = follow_ | follow_2-{'#'}
                            follow_ = follow_ | follow(nt)
                        else:
                            follow_ = follow_ | follow_2
    return follow_

In [ ]:
no_of_terminals=int(input("Enter no. of terminals: "))

terminals = ['$']

print("Enter the terminals :")
for _ in range(no_of_terminals):
    terminals.append(input())

no_of_non_terminals=int(input("Enter no of non terminals: "))

non_terminals = ['S']

print("Enter the non terminals :")
for _ in range(no_of_non_terminals-1):
    non_terminals.append(input())

starting_symbol = input("Enter the starting symbol: ")

no_of_productions = int(input("Enter no of productions: "))

productions = []

print("Enter the productions:")
for _ in range(no_of_productions):
    productions.append(input())

productions_dict = {}

for nT in non_terminals:
    productions_dict[nT] = []

for production in productions:
    nonterm_to_prod = production.split("->")
    alternatives = nonterm_to_prod[1].split("|")
    for alternative in alternatives:
        productions_dict[nonterm_to_prod[0]].append(alternative)

FIRST = {}
FOLLOW = {}
for non_terminal in non_terminals:
    FIRST[non_terminal] = set()

for non_terminal in non_terminals:
    FOLLOW[non_terminal] = set()

for non_terminal in non_terminals:
    FIRST[non_terminal] = FIRST[non_terminal] | first(non_terminal)

FOLLOW[starting_symbol] = FOLLOW[starting_symbol] | {'$'}
for non_terminal in non_terminals:
    FOLLOW[non_terminal] = FOLLOW[non_terminal] | follow(non_terminal)

print("{: ^20}{: ^20}{: ^20}".format('Non Terminals','First','Follow'))
for non_terminal in non_terminals:
    print("{: ^20}{: ^20}{: ^20}".format(non_terminal,str(FIRST[non_terminal]),str(FOLLOW[non_terminal])))
# S->AB|C
# A->a|b|#
# B->p|#
# C->c

Enter no. of terminals: 4
Enter the terminals :
a
b
c
p
Enter no of non terminals: 4
Enter the non terminals :
A
B
C
Enter the starting symbol: S
Enter no of productions: 4
Enter the productions:
S->AB|C
A->a|b|#
B->p|#
C->c
   Non Terminals           First               Follow       
         S          {'b', '#', 'c', 'p', 'a'}       {'$'}        
         A            {'b', '#', 'a'}        {'$', 'p'}     
         B               {'#', 'p'}            {'$'}        
         C                 {'c'}               {'$'}        


In [ ]:
def solve_table():
    ll1_table = {}
    for non_terminal in non_terminals:
        ll1_table[non_terminal] = {}
        for terminal in terminals:
            ll1_table[non_terminal][terminal] = []

    for non_terminal, productions in productions_dict.items():
        for production in productions:
            first_set = first(production)
            for terminal in first_set:
                if terminal != '#':
                    ll1_table[non_terminal][terminal].append(production)
            if '#' in first_set:
                for terminal in FOLLOW[non_terminal]:
                    ll1_table[non_terminal][terminal].append(production)
            if '#' in first_set and '$' in FOLLOW[non_terminal]:
                ll1_table[non_terminal]['$'].append(production)
    return ll1_table

def solve(ll1_table):
    for non_terminal, row in ll1_table.items():
        print(f"{non_terminal} -> {row}")

table = solve_table()
solve(table)

S -> {'$': ['AB', 'AB'], 'a': ['AB'], 'b': ['AB'], 'c': ['C'], 'p': ['AB']}
A -> {'$': ['#', '#'], 'a': ['a'], 'b': ['b'], 'c': [], 'p': ['#']}
B -> {'$': ['#', '#'], 'a': [], 'b': [], 'c': [], 'p': ['p']}
C -> {'$': [], 'a': [], 'b': [], 'c': ['c'], 'p': []}


In [ ]:
start_symbol = input("Enter the start symbol: ")
string = input("Enter the string to be validated: ")

stack = [start_symbol+'$']
buffer = [(string+'$')[::-1]]

print(f"String reverssd: {buffer[0]}")

parsing_table = {
    # follow for epshilon
  'S': [('$', '#'), ('a', 'AB'), ('p', 'AB'), ('b', 'AB'), ('c', 'C')],
  'A': [('$', '#'), ('b', 'b'), ('p', ' '), ('a', 'a'), ('c', ' ')],
  'B': [('$', '#'), ('b', ' '), ('c', ' '), ('a', ' '), ('p', 'p')],
  'C': [('$', ' '), ('p', ' '), ('b', ' '), ('c', 'c'), ('a', ' ')]
}

def Rule(string, start_symbol):
  if start_symbol not in parsing_table:
    return []
  i = parsing_table[start_symbol]
  for j, k in i:
    if j == string.strip():
      return [j, k]
  return []

def solve():
  string = list(buffer.pop())
  string1 = list(stack.pop())
  while len(string):
    top = string[-1]
    top1 = string1[0]
    if top == top1:
      string.pop()
      string1.pop(0)
      print(string, string1)
      continue
    rule = Rule(top, top1)
    if not rule:
      print("False")
      return 0
    print(f"Rule: {top1}->{rule[1]}")
    string1.pop(0)
    string1 = list(rule[1]) + string1
    print(string, string1)

  return 1
# Examples: ap,c,abnot
if solve():
  print("String Valid")
else:
  print("String not valid")

Enter the start symbol: S
Enter the string to be validated: ap
String reverssd: $pa
Rule: S->AB
['$', 'p', 'a'] ['A', 'B', '$']
Rule: A->a
['$', 'p', 'a'] ['a', 'B', '$']
['$', 'p'] ['B', '$']
Rule: B->p
['$', 'p'] ['p', '$']
['$'] ['$']
[] []
String Valid


In [ ]:
start_symbol = input("Enter the start symbol: ")
string = input("Enter the string to be validated: ")

stack = [start_symbol+'$']
buffer = [(string+'$')[::-1]]

print(f"String reverssd: {buffer[0]}")

parsing_table = {
    # follow for epshilon
  'S': [('$', '#'), ('a', 'AB'), ('p', 'AB'), ('b', 'AB'), ('c', 'C')],
  'A': [('$', '#'), ('b', 'b'), ('p', ' '), ('a', 'a'), ('c', ' ')],
  'B': [('$', '#'), ('b', ' '), ('c', ' '), ('a', ' '), ('p', 'p')],
  'C': [('$', ' '), ('p', ' '), ('b', ' '), ('c', 'c'), ('a', ' ')]
}

def Rule(string, start_symbol):
  if start_symbol not in parsing_table:
    return []
  i = parsing_table[start_symbol]
  for j, k in i:
    if j == string.strip():
      return [j, k]
  return []

def solve():
  string = list(buffer.pop())
  string1 = list(stack.pop())
  while len(string):
    top = string[-1]
    top1 = string1[0]
    if top == top1:
      string.pop()
      string1.pop(0)
      print(string, string1)
      continue
    rule = Rule(top, top1)
    if not rule:
      print("False")
      return 0
    print(f"Rule: {top1}->{rule[1]}")
    string1.pop(0)
    string1 = list(rule[1]) + string1
    print(string, string1)

  return 1
# Examples: ap,c,abnot
if solve():
  print("String Valid")
else:
  print("String not valid")

Enter the start symbol: S
Enter the string to be validated: abc
String reverssd: $cba
Rule: S->AB
['$', 'c', 'b', 'a'] ['A', 'B', '$']
Rule: A->a
['$', 'c', 'b', 'a'] ['a', 'B', '$']
['$', 'c', 'b'] ['B', '$']
Rule: B-> 
['$', 'c', 'b'] [' ', '$']
False
String not valid
